<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


<div style="text-align: center;">
  <h1>Empirical Asset Pricing with ML in Europe</h1>
  <h3>HEC Liege</h3>
  <h4><em>Lucas Dubois and Myriam Lamborelle</em></h4>
</div>

<hr style="border:none;height:6px;background:#fff;margin:1em 0;">


In [612]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
import torch
import torch.nn as nn
import torch.optim as optim
from dateutil.relativedelta import relativedelta
from statsmodels.stats import diagnostic
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.stats.sandwich_covariance import se_cov
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from stargazer.stargazer import Stargazer
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.formula.api as smf
import warnings
import re

In [309]:
path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/DATA/"

In [310]:
figure_path="/Users/lucasdubois/Desktop/LaTeX/EAP-ML/CODE/Images/"

In [311]:
offwhite = (230/255, 230/255, 220/255)
midnight = (0/255, 22/255, 36/255)
steelblue = (171/255, 193/255, 223/255)
primaryred = (127/255, 20/255, 22/255)
harmonizedblue =(48/255,88/255,140/255)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>1</small>&nbsp;&nbsp;&nbsp;&nbsp;Database:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">








<div style="text-align: align;">
  <h3> <small>1.1</small>&nbsp;&nbsp;&nbsp;&nbsp;Stocks:</h3>
</div>

In [316]:
df = pd.read_excel(path + "stoxx600.xlsx")

df.rename(columns={df.columns[0]: "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [317]:
df = df.loc[:, ~df.columns.str.startswith("#ERROR")]

In [318]:
df_long = df.melt(
    id_vars="Date",
    var_name="col",
    value_name="value"
)

In [319]:
df_long[["Stock", "Datatype"]] = (
    df_long["col"]
    .str.split(" - ", n=1, expand=True)
)

In [320]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Datatype",
        values="value"
    )
    .reset_index()
)


In [321]:
df_panel = df_panel.sort_values(["Stock", "Date"])

print(df_panel.head())
print(df_panel.tail())
print(df_panel.info())


Datatype       Date     Stock  DIVIDEND YIELD  MARKET VALUE  \
0        2005-01-01  3I GROUP            2.13       4085.77   
435      2005-02-01  3I GROUP            2.00       4355.70   
871      2005-03-01  3I GROUP            2.04       4279.01   
1307     2005-04-01  3I GROUP            2.11       4134.96   
1743     2005-05-01  3I GROUP            2.23       3913.99   

Datatype  MRKT VALUE TO BOOK   PER  PRICE INDEX  TOT RETURN IND  \
0                       1.12  30.4        244.9          302.56   
435                     1.20  32.4        261.0          322.55   
871                     1.18  31.8        256.4          316.87   
1307                    1.03  30.7        247.4          305.74   
1743                    0.98  29.1        234.6          289.84   

Datatype  TURNOVER BY VALUE  TURNOVER BY VOLUME  
0                  340749.3             83500.9  
435                338096.4             81321.8  
871                446679.1            100759.9  
1307              

In [322]:
df_panel["Stock"].nunique()

599

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.2</small>&nbsp;&nbsp;&nbsp;&nbsp;Risk-Free:</h3>
</div>

In [325]:
rf_df = pd.read_excel(path + "EURIBOR.xlsx", header=None)

In [326]:
rf_df.columns = rf_df.iloc[0]
rf_df = rf_df.iloc[1:].reset_index(drop=True)


In [327]:
rf_df.head()
rf_df.columns


Index(['DATE', 'TIME PERIOD', 'OBS.VALUE'], dtype='object', name=0)

In [328]:
rf_df = rf_df.rename(columns={
    "DATE": "Date",
    "OBS.VALUE": "Rf"
})
rf_df = rf_df.drop(columns=["TIME PERIOD"])

In [329]:
rf_df["Date"] = pd.to_datetime(rf_df["Date"])
rf_df["Rf"] = rf_df["Rf"].astype(float) / 100 / 12

In [330]:
rf_df["Date"] = rf_df["Date"] + pd.offsets.MonthBegin(1)


In [331]:
rf_df.head

<bound method NDFrame.head of 0         Date        Rf
0   2005-01-01  0.001806
1   2005-02-01  0.001759
2   2005-03-01  0.001753
3   2005-04-01  0.001753
4   2005-05-01  0.001754
..         ...       ...
247 2025-08-01  0.001577
248 2025-09-01  0.001575
249 2025-10-01  0.001581
250 2025-11-01  0.001589
251 2025-12-01  0.001588

[252 rows x 2 columns]>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>1.3</small>&nbsp;&nbsp;&nbsp;&nbsp;Merging:</h3>
</div>

In [334]:
df_almost = df_panel.merge(
    rf_df,
    on="Date",
    how="left"
)

In [335]:
df_almost["Rf"].describe()

count    133070.000000
mean          0.000881
std           0.001363
min          -0.000497
25%          -0.000308
50%           0.000216
75%           0.001866
max           0.004026
Name: Rf, dtype: float64

In [336]:
df_almost.to_csv(path + "Data1.csv", index=False)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>2</small>&nbsp;&nbsp;&nbsp;&nbsp;Data Management:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.1</small>&nbsp;&nbsp;&nbsp;&nbsp;From Long to Panel</h3>
</div>

In [341]:
cols = df_almost.columns.tolist()
cols_no_date = [c for c in cols if c != "Date"]


In [342]:
df_long = df_almost.melt(
    id_vars=["Date", "Stock"],
    var_name="Variable",
    value_name="value"
)

In [343]:
df_long.columns


Index(['Date', 'Stock', 'Variable', 'value'], dtype='object')

In [344]:
df_long.columns
df_long.head()

,Date,Stock,Variable,value
0,2005-01-01,3I GROUP,DIVIDEND YIELD,2.13
1,2005-02-01,3I GROUP,DIVIDEND YIELD,2.00
2,2005-03-01,3I GROUP,DIVIDEND YIELD,2.04
3,2005-04-01,3I GROUP,DIVIDEND YIELD,2.11
4,2005-05-01,3I GROUP,DIVIDEND YIELD,2.23


In [345]:
df_panel = (
    df_long
    .pivot_table(
        index=["Date", "Stock"],
        columns="Variable",
        values="value"
    )
    .reset_index()
)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.2.</small>&nbsp;&nbsp;&nbsp;&nbsp;Excess Returns:</h3>
</div>

In [348]:
df_panel["RET"] = (
    df_panel
    .groupby("Stock")["TOT RETURN IND"]
    .pct_change(fill_method=None)
)

In [349]:
df_panel["RET_FWD"] = df_panel.groupby("Stock")["RET"].shift(-1)
df_panel["EXCESS_RET_FWD"] = df_panel["RET_FWD"] - df_panel["Rf"]


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.3</small>&nbsp;&nbsp;&nbsp;&nbsp;Firm Characteristics:</h3>
</div>

In [352]:
def safe_log(x):
    out = np.full_like(x, np.nan, dtype="float64")
    mask = x > 0
    out[mask] = np.log(x[mask])
    return out

In [353]:
df_panel["TURN_VAL"]  = safe_log(df_panel["TURNOVER BY VALUE"])
df_panel["TURN_VOL"]  = safe_log(df_panel["TURNOVER BY VOLUME"])
df_panel["LOG_PRICE"] = safe_log(df_panel["PRICE INDEX"])
df_panel["SIZE"]      = safe_log(df_panel["MARKET VALUE"])
df_panel["BM"]        = 1 / df_panel["MRKT VALUE TO BOOK"]
df_panel["DY"]        = df_panel["DIVIDEND YIELD"] / 100
df_panel["EY"]        = 1 / df_panel["PER"]

df_panel["MOM3"] = (
    df_panel.groupby("Stock")["LOG_PRICE"]
            .apply(lambda s: s.shift(2) - s.shift(3))
            .reset_index(level=0, drop=True)
)

df_panel["MOM12"] = (
    df_panel.groupby("Stock")["LOG_PRICE"]
            .apply(lambda s: s.shift(2) - s.shift(12))
            .reset_index(level=0, drop=True)
)

df_panel["VOL12"] = (
    df_panel.groupby("Stock")["RET"]
            .rolling(12)
            .std()
            .reset_index(level=0, drop=True)
)


In [354]:
market_cols = [
    "SIZE","MOM3","MOM12","VOL12",
    "TURN_VAL","TURN_VOL","LOG_PRICE"
]

fund_cols = [
    "BM","DY","EY"
]

df_panel[market_cols] = df_panel.groupby("Stock")[market_cols].shift(1)

df_panel[fund_cols] = df_panel.groupby("Stock")[fund_cols].shift(6)


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.4</small>&nbsp;&nbsp;&nbsp;&nbsp;Interaction Terms:</h3>
</div>

In [357]:
df_panel["SIZE_BM"]   = df_panel["SIZE"] * df_panel["BM"]
df_panel["SIZE_MOM"]  = df_panel["SIZE"] * df_panel["MOM12"]
df_panel["SIZE_VOL"]  = df_panel["SIZE"] * df_panel["VOL12"]
df_panel["SIZE_LIQ"]  = df_panel["SIZE"] * df_panel["TURN_VAL"]
df_panel["BM_MOM"] = df_panel["BM"] * df_panel["MOM12"]
df_panel["BM_VOL"] = df_panel["BM"] * df_panel["VOL12"]
df_panel["BM_LIQ"] = df_panel["BM"] * df_panel["TURN_VAL"]
df_panel["MOM_VOL"] = df_panel["MOM12"] * df_panel["VOL12"]
df_panel["MOM_LIQ"] = df_panel["MOM12"] * df_panel["TURN_VAL"]
df_panel["MOM_PRICE"] = df_panel["MOM12"] * df_panel["LOG_PRICE"]
df_panel["EY_MOM"] = df_panel["EY"] * df_panel["MOM12"]
df_panel["DY_SIZE"] = df_panel["DY"] * df_panel["SIZE"]

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>2.5</small>&nbsp;&nbsp;&nbsp;&nbsp;Standarization & Winsorization :</h3>
</div>

In [360]:
characteristics = [
    "SIZE","BM","DY","EY","MOM3","MOM12","VOL12",
    "TURN_VAL","TURN_VOL","LOG_PRICE"
]

interaction_terms = [
    "SIZE_BM","SIZE_MOM","SIZE_VOL","SIZE_LIQ",
    "BM_MOM","BM_VOL","BM_LIQ",
    "MOM_VOL","MOM_LIQ","MOM_PRICE",
    "EY_MOM","DY_SIZE"
]

predictors = characteristics + interaction_terms

In [361]:
print(df_panel.head())

Variable       Date                Stock  DIVIDEND YIELD  MARKET VALUE  \
0        2005-01-01             3I GROUP            2.13       4085.77   
1        2005-01-01  A P MOLLER MAERSK B            0.66     100668.40   
2        2005-01-01                  A2A            2.94       3060.08   
3        2005-01-01             AALBERTS            1.54        878.29   
4        2005-01-01            ABB LTD N            0.00      13477.75   

Variable  MRKT VALUE TO BOOK   PER  PRICE INDEX        Rf  TOT RETURN IND  \
0                       1.12  30.4        244.9  0.001806          302.56   
1                       1.75   9.1       5126.9  0.001806         6348.85   
2                       1.86  15.7        199.7  0.001806          231.36   
3                       2.94  20.4       1378.8  0.001806         1844.24   
4                       2.93  10.4        292.8  0.001806          579.90   

Variable  TURNOVER BY VALUE  ...  SIZE_VOL  SIZE_LIQ  BM_MOM  BM_VOL  BM_LIQ  \
0           

In [362]:
print(f'We have {len(predictors)} ({len(characteristics)} characteristics and {len(interaction_terms)} interaction terms)')

We have 22 (10 characteristics and 12 interaction terms)


But first, following the GKN

In [364]:
class CSStandardizer:
    def __init__(self, lower=0.01, upper=0.99):
        self.lower = lower
        self.upper = upper
        self.q_low = None
        self.q_high = None
        self.mean = None
        self.std = None

    def fit(self, df, cols):
        self.q_low  = df[cols].quantile(self.lower)
        self.q_high = df[cols].quantile(self.upper)
        self.mean   = df[cols].mean()
        self.std    = df[cols].std()
        return self

    def transform(self, df, cols):
        x = df[cols].clip(self.q_low, self.q_high, axis=1)
        z = (x - self.mean) / self.std

        return z.fillna(0.0)


In [365]:
df_FINAL= df_panel

In [366]:
df_FINAL.to_csv(path + "FINAL.csv", index=False)

<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>3</small>&nbsp;&nbsp;&nbsp;&nbsp;Splitting:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">

<div style="text-align: align;">
  <h3> <small>3.1</small>&nbsp;&nbsp;&nbsp;&nbsp;Test-Train Splitting:</h3>
</div>

In [371]:
df_FINAL["Date"] = pd.to_datetime(df_FINAL["Date"])

In [372]:
train = df_FINAL[df_FINAL["Date"] < "2019-01-01"].copy()
test  = df_FINAL[df_FINAL["Date"] >= "2019-01-01"].copy()

print(train["Date"].min(), train["Date"].max())
print(test["Date"].min(), test["Date"].max())

2005-01-01 00:00:00 2018-12-01 00:00:00
2019-01-01 00:00:00 2025-12-01 00:00:00


In [373]:
y_train = train["EXCESS_RET_FWD"]
y_test  = test["EXCESS_RET_FWD"]

In [374]:
X_train = train[predictors]
X_test  = test[predictors]

In [375]:
X_train.shape, y_train.shape

((84217, 22), (84217,))

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>3.2</small>&nbsp;&nbsp;&nbsp;&nbsp; Exluding NaN for Returns & Imputation :</h3>
</div>

In [378]:
train = train.dropna(subset=["EXCESS_RET_FWD"])
test  = test.dropna(subset=["EXCESS_RET_FWD"])

In [379]:
def cs_median_impute(df, cols):
    df[cols] = (
        df
        .groupby("Date")[cols]
        .transform(lambda x: x.fillna(x.median()))
    )
    return df


In [380]:
train = cs_median_impute(train, predictors)
test  = cs_median_impute(test, predictors)


In [381]:
missing_by_date = (
    train
    .groupby("Date")[predictors]
    .apply(lambda x: x.isna().any().any())
)

missing_by_date.value_counts()


False    155
True      13
Name: count, dtype: int64

In [382]:
valid_dates = missing_by_date[~missing_by_date].index

train = train[train["Date"].isin(valid_dates)].copy()


In [383]:
train[predictors].isna().sum().sum(), train["EXCESS_RET_FWD"].isna().sum()


(0, 0)

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>3.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Rolling Window Cross-Validation:</h3>
</div>

In [386]:
class RollingPanelSplit:
    """
    Rolling time-based split for panel data (Date × Stock)

    Parameters
    ----------
    train_months : int
        Length of training window in months
    val_months : int or None
        Length of validation window in months (None = no validation)
    test_months : int
        Length of test window in months
    step_months : int
        Step size of the rolling window
    """

    def __init__(self,
                 train_months=120,
                 val_months=None,
                 test_months=12,
                 step_months=12):

        self.train_months = train_months
        self.val_months = val_months
        self.test_months = test_months
        self.step_months = step_months

    def split(self, df, date_col="Date"):
        """
        Returns a list of (train_idx, val_idx, test_idx)
        """

        df = df.sort_values(date_col).copy()
        dates = df[date_col].drop_duplicates().sort_values()

        splits = []

        start = dates.min() + relativedelta(months=self.train_months)

        while True:
            train_start = start - relativedelta(months=self.train_months)
            train_end = start

            if self.val_months is not None:
                val_end = train_end + relativedelta(months=self.val_months)
                test_end = val_end + relativedelta(months=self.test_months)
            else:
                val_end = None
                test_end = train_end + relativedelta(months=self.test_months)

            if test_end > dates.max():
                break

            train_idx = df[
                (df[date_col] >= train_start) &
                (df[date_col] < train_end)
            ].index

            if self.val_months is not None:
                val_idx = df[
                    (df[date_col] >= train_end) &
                    (df[date_col] < val_end)
                ].index
            else:
                val_idx = None

            test_idx = df[
                (df[date_col] >= (val_end if val_end else train_end)) &
                (df[date_col] < test_end)
            ].index

            splits.append((train_idx, val_idx, test_idx))

            start += relativedelta(months=self.step_months)

        return splits


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>4</small>&nbsp;&nbsp;&nbsp;&nbsp;Indicators Functions:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.1</small>&nbsp;&nbsp;&nbsp;&nbsp; $R^2_{OOS}$ &nbsp;Function:</h3>
</div>

In [391]:
def oos_r2(df, y_true="y_true", y_pred="y_pred", date_col="Date"):
    df = df.copy()
    df = df.sort_values(date_col)

    m = df.groupby(date_col)[y_true].mean()
    m_hist = m.expanding().mean().shift(1)

    df["mean_fc"] = df[date_col].map(m_hist)

    valid = (
        df["mean_fc"].notna()
        & df[y_true].notna()
        & df[y_pred].notna()
    )

    if valid.sum() == 0:
        return np.nan

    sse = ((df.loc[valid, y_true] - df.loc[valid, y_pred]) ** 2).sum()
    sst = ((df.loc[valid, y_true] - df.loc[valid, "mean_fc"]) ** 2).sum()

    if not np.isfinite(sst) or sst <= 0:
        return np.nan

    return 1 - sse / sst



<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.2</small>&nbsp;&nbsp;&nbsp;&nbsp; H-L Portifolio Function:</h3>
</div>

In [394]:
def hl_month(df, q=0.1, pred_col="y_pred", ret_col="y_true"):
    """
    High-Low portfolio return for one cross-section
    """
    df = df[[pred_col, ret_col]].dropna()
    if df.empty:
        return np.nan

    q_low = df[pred_col].quantile(q)
    q_high = df[pred_col].quantile(1 - q)

    return (
        df.loc[df[pred_col] >= q_high, ret_col].mean()
        - df.loc[df[pred_col] <= q_low, ret_col].mean()
    )



<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>4.2</small>&nbsp;&nbsp;&nbsp;&nbsp; $Sharpe$ (H-L) Function:</h3>
</div>

In [397]:
def sharpe(returns, periods_per_year=12):
    mean = returns.mean()
    std  = returns.std()
    return np.sqrt(periods_per_year) * mean / std



In [398]:
def sharpe_gkn(returns, oos_r2, periods_per_year=12):
    SR = sharpe(returns, periods_per_year)
    return np.sqrt((SR**2 + oos_r2) / (1 - oos_r2))


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>5</small>&nbsp;&nbsp;&nbsp;&nbsp;OLS:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>5.1</small>&nbsp;&nbsp;&nbsp;&nbsp; Function:</h3>
</div>

In [403]:
splitter = RollingPanelSplit(
    train_months=120,   
    val_months=None,    
    test_months=12,    
    step_months=12      
)
splits = splitter.split(df_FINAL, date_col="Date")


In [404]:
def run_rolling_ols(df, splits, predictors, target):
    """
    Rolling OLS with proper lagging and train-only standardization
    """
    oos_results = []

    for split_id, (train_idx, _, test_idx) in enumerate(splits):

        train = df.loc[train_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Winsorazation
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        X_test  = stdzr.transform(test, predictors).values

        y_train = train[target].values
        y_test  = test[target].values

        # ----------------------------
        # OLS FULL
        # ----------------------------
        model = LinearRegression(fit_intercept=True)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "OLS"
        out["split"]  = split_id

        oos_results.append(out)

    return pd.concat(oos_results, ignore_index=True)


In [405]:
target = "EXCESS_RET_FWD"


In [406]:
ols_full = run_rolling_ols(
    df=df_FINAL,
    splits=splits,
    predictors=predictors,
    target=target
)


In [407]:
ols_full.columns


Index(['Date', 'Stock', 'y_true', 'y_pred', 'model', 'split'], dtype='object', name='Variable')

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>5.2</small>&nbsp;&nbsp;&nbsp;&nbsp; Results:</h3>
</div>

In [410]:
r2_ols = oos_r2(ols_full)
print(f"OOS R² (OLS): {r2_ols:.4f}")


OOS R² (OLS): 0.0023


In [411]:
hl_ols_full = (
    ols_full
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

In [415]:
sharpe_ols_full = sharpe_gkn(hl_ols_full, r2_ols,periods_per_year=12)

print(f"SHARPE (GKN) (OLS H–L): {sharpe_ols_full:.2f}")

SHARPE (GKN) (OLS H–L): 0.88


In [417]:
ols_full.groupby("Date")["y_pred"].std().describe()


count    120.000000
mean       0.007938
std        0.001713
min        0.004044
25%        0.006837
50%        0.007625
75%        0.008862
max        0.012917
Name: y_pred, dtype: float64

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>5.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Look-Ahead-Bias Check:</h3>
</div>

In [421]:
tmp = ols_full.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


Placebo Sharpe: 0.11120857056872947


In [423]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        ols_full.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))


0.1 0.8770983471262924
0.2 0.678040788032061
0.3 0.6590453440994605
0.4 0.601687629319382


In [425]:
ols_test = ols_full.copy()
ols_test["y_pred"] = ols_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    ols_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))


Shifted Sharpe: 1.2987902677590761


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>6</small>&nbsp;&nbsp;&nbsp;&nbsp; OLS-3:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>6.1</small>&nbsp;&nbsp;&nbsp;&nbsp; Function:</h3>
</div>

In [431]:
predictors_3 = ["SIZE", "BM", "MOM12"]


In [433]:
def run_rolling_ols(df, splits, predictors, target):
    """
    Rolling OLS-3 with proper lagging and train-only standardization
    """
    oos_results = []

    for split_id, (train_idx, _, test_idx) in enumerate(splits):

        train = df.loc[train_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Winsorazation
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        X_test  = stdzr.transform(test, predictors).values

        y_train = train[target].values
        y_test  = test[target].values

        # ----------------------------
        # OLS FULL
        # ----------------------------
        model = LinearRegression(fit_intercept=True)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "OLS"
        out["split"]  = split_id

        oos_results.append(out)

    return pd.concat(oos_results, ignore_index=True)


In [435]:
target = "EXCESS_RET_FWD"


In [437]:
ols_3 = run_rolling_ols(
    df=df_FINAL,
    splits=splits,
    predictors=predictors_3,
    target=target
)


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>6.2</small>&nbsp;&nbsp;&nbsp;&nbsp; Results:</h3>
</div>

In [441]:
r2_ols3 = oos_r2(ols_3)
print(f"OOS R² (OLS): {r2_ols3:.4f}")


OOS R² (OLS): 0.0092


In [443]:
hl_ols_3 = (
    ols_3
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

In [445]:
sharpe_ols_3= sharpe_gkn(hl_ols_3, r2_ols3,periods_per_year=12)

print(f"SHARPE (GKN) (OLS-3 H–L): {sharpe_ols_3:.2f}")


SHARPE (GKN) (OLS-3 H–L): 0.86


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>6.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Look-Ahead-Bias Check:</h3>
</div>

In [449]:
tmp = ols_3.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


Placebo Sharpe: 0.21531032773612316


In [451]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        ols_3.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))


0.1 0.8477096700770961
0.2 0.7032105582530832
0.3 0.6953530530282129
0.4 0.6948904413430819


In [453]:
ols_test = ols_3.copy()
ols_test["y_pred"] = ols_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    ols_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))


Shifted Sharpe: 0.8477255661036559


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>7</small>&nbsp;&nbsp;&nbsp;&nbsp; Elastic-Net:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>7.1</small>&nbsp;&nbsp;&nbsp;&nbsp; Function:</h3>
</div>

In [459]:
splitter = RollingPanelSplit(
    train_months=120,
    val_months=12,   
    test_months=12,
    step_months=12
)

splits = splitter.split(df_FINAL, date_col="Date")


In [461]:
alpha_grid = np.logspace(-4, 1, 20)
l1_ratio = 0.5  #FOR ELASTIC NET!

In [463]:
def run_rolling_enet(df, splits, predictors, target, alpha_grid, l1_ratio=0.5):
    oos_results = []

    for split_id, (train_idx, val_idx, test_idx) in enumerate(splits):

        # ----------------------------
        # Split data
        # ----------------------------
        train = df.loc[train_idx].copy()
        val   = df.loc[val_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        val   = val.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(val) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Standardization (TRAIN ONLY)
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        y_train = train[target].values

        X_val = stdzr.transform(val, predictors).values
        y_val = val[target].values

        X_test = stdzr.transform(test, predictors).values
        y_test = test[target].values

        # ----------------------------
        # Hyperparameter tuning (TRAIN → VAL)
        # ----------------------------
        best_alpha = None
        best_mse = np.inf

        for alpha in alpha_grid:
            model = ElasticNet(
                alpha=alpha,
                l1_ratio=l1_ratio,
                fit_intercept=True,
                max_iter=10_000
            )
            model.fit(X_train, y_train)
            val_pred = model.predict(X_val)
            mse = np.mean((y_val - val_pred) ** 2)

            if mse < best_mse:
                best_mse = mse
                best_alpha = alpha

        # ----------------------------
        # Refit on TRAIN ∪ VAL
        # ----------------------------
        X_tv = np.vstack([X_train, X_val])
        y_tv = np.concatenate([y_train, y_val])

        final_model = ElasticNet(
            alpha=best_alpha,
            l1_ratio=l1_ratio,
            fit_intercept=True,
            max_iter=10_000
        )
        final_model.fit(X_tv, y_tv)

        # ----------------------------
        # Predict on TEST
        # ----------------------------
        y_pred = final_model.predict(X_test)

        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "ElasticNet"
        out["alpha"]  = best_alpha
        out["split"]  = split_id

        oos_results.append(out)

    return pd.concat(oos_results, ignore_index=True)


In [465]:
enet_full = run_rolling_enet(
    df=df_FINAL,
    splits=splits,
    predictors=predictors,
    target="EXCESS_RET_FWD",
    alpha_grid=alpha_grid,
    l1_ratio=0.5
)


In [466]:
enet_full["alpha"].describe()
enet_full.groupby("Date")["y_pred"].std().describe()


count    108.000000
mean       0.003426
std        0.003118
min        0.000000
25%        0.000000
50%        0.003639
75%        0.006458
max        0.008744
Name: y_pred, dtype: float64

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>7.2</small>&nbsp;&nbsp;&nbsp;&nbsp; Results:</h3>
</div>

In [516]:
r2_enet = oos_r2(enet_full)
print(f"OOS R² (ENet): {r2_enet:.4f}")


OOS R² (ENet): 0.0041


In [473]:
hl_enet = (
    enet_full
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(lambda m: hl_month(m, q=0.1))
)


In [475]:
sharpe_enet = sharpe_gkn(hl_enet, r2_enet,periods_per_year=12)

print(f"SHARPE (GKN) (OLS H–L): {sharpe_enet:.2f}")

SHARPE (GKN) (OLS H–L): 0.29


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>7.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Look-Ahead-Bias Check:</h3>
</div>

In [479]:
tmp = enet_full.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


Placebo Sharpe: 0.7530584761816607


In [638]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        enet_full.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))


0.1 0.2813170610346764
0.2 0.2394610472971748
0.3 0.21212771800992192
0.4 0.16361623412188916


In [483]:
enet_test = enet_full.copy()
enet_test["y_pred"] = enet_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    enet_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))


Shifted Sharpe: 0.7557518462481617


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>8</small>&nbsp;&nbsp;&nbsp;&nbsp; Random Forest:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>8.1</small>&nbsp;&nbsp;&nbsp;&nbsp; Function:</h3>
</div>

In [489]:
splitter = RollingPanelSplit(
    train_months=120,
    val_months=12,
    test_months=12,
    step_months=12
)

splits = splitter.split(df_FINAL, date_col="Date")


In [491]:
param_grid1 = {
    "max_depth": [1,2,3,4,5,6],
    "max_features": [3, 6, 12, 24, 46, 49]
}


In [504]:
param_grid = {
    "max_depth": [2, 4, 6],
    "max_features": [int(np.sqrt(len(predictors))), len(predictors)//3]
}

grid = list(ParameterGrid(param_grid))


In [506]:
grid = list(ParameterGrid(param_grid))

In [586]:
from sklearn.ensemble import RandomForestRegressor

def run_rolling_rf_gkn_style(df, splits, predictors, target, grid, random_state=42):

    predictions = []
    y_test_list = []
    dates = []
    dic_r2_all = {}
    dic_max_depth_all = {}

    for split_id, (train_idx, val_idx, test_idx) in enumerate(splits):

        # ----------------------------
        # Split data
        # ----------------------------
        train = df.loc[train_idx].copy()
        val   = df.loc[val_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        val   = val.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(val) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Standardization (TRAIN ONLY)
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        y_train = train[target].values

        X_val = stdzr.transform(val, predictors).values
        y_val = val[target].values

        X_test = stdzr.transform(test, predictors).values
        y_test = test[target].values

        # ----------------------------
        # Hyperparameter tuning (TRAIN → VAL)
        # ----------------------------
        mse = np.full(len(grid), np.nan, dtype=np.float64)

        for j, params in enumerate(grid):

            model = RandomForestRegressor(
                n_estimators=100,
                max_depth=params["max_depth"],
                max_features=params["max_features"],
                min_samples_leaf=40,
                n_jobs=-1,
                random_state=random_state
            )

            model.fit(X_train, y_train)
            val_pred = model.predict(X_val)
            mse[j] = np.mean((y_val - val_pred) ** 2)

        best_idx = np.argmin(mse)
        best_params = grid[best_idx]

        dic_max_depth_all[split_id] = best_params["max_depth"]

        # ----------------------------
        # Refit on TRAIN ∪ VAL
        # ----------------------------
        X_tv = np.vstack([X_train, X_val])
        y_tv = np.concatenate([y_train, y_val])

        final_model = RandomForestRegressor(
            n_estimators=200,
            max_depth=best_params["max_depth"],
            max_features=best_params["max_features"],
            min_samples_leaf=40,
            n_jobs=-1,
            random_state=random_state
        )

        final_model.fit(X_tv, y_tv)

        y_pred = final_model.predict(X_test)

        # ----------------------------
        # Store results
        # ----------------------------
        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "RF"
        out["split"]  = split_id
        out["max_depth"] = best_params["max_depth"]
        out["max_features"] = best_params["max_features"]

        predictions.append(out)

    return pd.concat(predictions, ignore_index=True), dic_max_depth_all


In [588]:
rf, rf_depths = run_rolling_rf_gkn_style(
    df=df_FINAL,
    splits=splits,
    predictors=predictors,
    target="EXCESS_RET_FWD",
    grid=grid
)


In [589]:
r2_rf = oos_r2(rf)
print(f"OOS R² (RF): {r2_rf:.4f}")

OOS R² (RF): 0.0102


In [590]:
hl_rf = (
    rf
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(lambda m: hl_month(m, q=0.1))
)


In [591]:
sharpe_rf = sharpe_gkn(hl_rf, r2_rf,periods_per_year=12)

print(f"SHARPE (GKN) (RF): {sharpe_rf:.2f}")

SHARPE (GKN) (RF): 0.70


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>7.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Look-Ahead-Bias Check:</h3>
</div>

In [636]:
tmp = rf.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


Placebo Sharpe: 0.32345133089142203


In [640]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        rf.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))

0.1 0.686320702191577
0.2 0.6222720661583901
0.3 0.5957072182406385
0.4 0.6396206858892465


In [642]:
rf_test = rf.copy()
rf_test["y_pred"] = rf_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    rf_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))


Shifted Sharpe: 0.9599599941744709


<hr style="border:none;height:4px;background:#fff;margin:1em 0;">


<div style="text-align: align;">
  <h2> <small>9</small>&nbsp;&nbsp;&nbsp;&nbsp; Neural Networks:</h2>
</div>

<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>9.1</small>&nbsp;&nbsp;&nbsp;&nbsp; Function:</h3>
</div>

In [606]:
nn_grid = [
    {"hidden_units": 16, "l2": 1e-4},
    {"hidden_units": 32, "l2": 1e-4},
    {"hidden_units": 32, "l2": 1e-3},
    {"hidden_units": 64, "l2": 1e-3},
]


In [614]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_units):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze()


In [616]:
def train_nn(X_train, y_train, X_val, y_val, hidden_units, l2,
             epochs=50, lr=1e-3, batch_size=1024, seed=42):

    torch.manual_seed(seed)

    model = SimpleNN(X_train.shape[1], hidden_units)

    optimizer = optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=l2   # ← L2 regularization
    )

    loss_fn = nn.MSELoss()

    X_tr = torch.tensor(X_train, dtype=torch.float32)
    y_tr = torch.tensor(y_train, dtype=torch.float32)
    X_va = torch.tensor(X_val, dtype=torch.float32)
    y_va = torch.tensor(y_val, dtype=torch.float32)

    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(X_tr)
        loss = loss_fn(pred, y_tr)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_pred = model(X_va)
        val_loss = loss_fn(val_pred, y_va).item()

    return model, val_loss


In [618]:
def run_rolling_nn(df, splits, predictors, target, nn_grid):

    oos_results = []

    for split_id, (train_idx, val_idx, test_idx) in enumerate(splits):

        print(f"NN split {split_id+1}/{len(splits)}")

        train = df.loc[train_idx].copy()
        val   = df.loc[val_idx].copy()
        test  = df.loc[test_idx].copy()

        train = train.dropna(subset=predictors + [target])
        val   = val.dropna(subset=predictors + [target])
        test  = test.dropna(subset=predictors + [target])

        if len(train) == 0 or len(val) == 0 or len(test) == 0:
            continue

        # ----------------------------
        # Standardization (TRAIN ONLY)
        # ----------------------------
        stdzr = CSStandardizer(lower=0.01, upper=0.99)
        stdzr.fit(train, predictors)

        X_train = stdzr.transform(train, predictors).values
        y_train = train[target].values

        X_val = stdzr.transform(val, predictors).values
        y_val = val[target].values

        X_test = stdzr.transform(test, predictors).values
        y_test = test[target].values

        # ----------------------------
        # Hyperparameter tuning
        # ----------------------------
        best_loss = np.inf
        best_spec = None

        for spec in nn_grid:
            _, val_loss = train_nn(
                X_train, y_train,
                X_val, y_val,
                hidden_units=spec["hidden_units"],
                l2=spec["l2"]
            )

            if val_loss < best_loss:
                best_loss = val_loss
                best_spec = spec

        # ----------------------------
        # Refit on TRAIN ∪ VAL
        # ----------------------------
        X_tv = np.vstack([X_train, X_val])
        y_tv = np.concatenate([y_train, y_val])

        final_model, _ = train_nn(
            X_tv, y_tv,
            X_val, y_val,      # dummy, unused
            hidden_units=best_spec["hidden_units"],
            l2=best_spec["l2"]
        )

        # ----------------------------
        # Predict on TEST
        # ----------------------------
        final_model.eval()
        with torch.no_grad():
            y_pred = final_model(
                torch.tensor(X_test, dtype=torch.float32)
            ).numpy()

        out = test[["Date", "Stock"]].copy()
        out["y_true"] = y_test
        out["y_pred"] = y_pred
        out["model"]  = "NN"
        out["hidden_units"] = best_spec["hidden_units"]
        out["l2"] = best_spec["l2"]
        out["split"] = split_id

        oos_results.append(out)

    return pd.concat(oos_results, ignore_index=True)


In [620]:
nn= run_rolling_nn(
    df=df_FINAL,
    splits=splits,
    predictors=predictors,
    target="EXCESS_RET_FWD",
    nn_grid=nn_grid
)


NN split 1/9
NN split 2/9
NN split 3/9
NN split 4/9
NN split 5/9
NN split 6/9
NN split 7/9
NN split 8/9
NN split 9/9


In [622]:
r2_nn = oos_r2(nn)
print(f"OOS R² (NNW): {r2_nn:.4f}")

OOS R² (NNW): -0.0961


In [624]:
hl_nn = (
    nn
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(lambda m: hl_month(m, q=0.1))
)

In [628]:
sharpe_nn = sharpe_gkn(hl_nn, r2_nn,periods_per_year=12)

print(f"SHARPE (GKN) (NNW): {sharpe_nn:.2f}")

SHARPE (GKN) (NNW): 1.09


<hr style="border:none; border-top:2px dashed #fff; margin:1em 0;">


<div style="text-align: align;">
  <h3> <small>7.3</small>&nbsp;&nbsp;&nbsp;&nbsp; Look-Ahead-Bias Check:</h3>
</div>

In [644]:
tmp = nn.copy()
tmp["y_pred"] = np.random.randn(len(tmp))

hl_placebo = (
    tmp.groupby("Date")[["y_pred", "y_true"]]
       .apply(hl_month)
)

print("Placebo Sharpe:", sharpe(hl_placebo))


Placebo Sharpe: -0.36928500040267964


In [648]:
for q in [0.1, 0.2, 0.3, 0.4]:
    hl_q = (
        nn.groupby("Date")[["y_pred", "y_true"]]
               .apply(lambda m: hl_month(m, q=q))
    )
    print(q, sharpe(hl_q))

0.1 1.1829894236449037
0.2 0.9343573672347858
0.3 1.0302374350889707
0.4 1.0143286507539206


In [650]:
nn_test = nn.copy()
nn_test["y_pred"] = nn_test.groupby("Stock")["y_pred"].shift(1)

hl_shifted = (
    nn_test
    .groupby("Date")[["y_pred", "y_true"]]
    .apply(hl_month)
)

print("Shifted Sharpe:", sharpe(hl_shifted))


Shifted Sharpe: 0.9388277475895077
